In [1]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from jupyter_plotly_dash import JupyterDash

from src.TransformData import dividir_datos, crear_df_error
from src.models import crear_linea_regresion_lineal, crear_linea_k_vecinos, crear_linea_regresion_logistica
from src.CreateFigures import crear_figuras_entrenamiento_prueba_validacion, crear_figuras_regresion_logistica, graficar_error

## Ejemplo de Regresión (Regresión Lineal)

### Cargar Datos

In [2]:
data_peso_altura = pd.read_csv('./data/weight-height.csv', nrows=400)

### Cargar aplicación

#### Comparación de ajuste entrenamiento, prueba, validación

In [3]:
app = JupyterDash(__name__)

# crear input
peso = data_peso_altura.Weight
altura = data_peso_altura.Height
datos_altura_peso = dividir_datos(peso, altura)

# app
app.layout = html.Div([html.Div([html.H1("Demo Regresión Lineal"),
                                 dcc.Dropdown(id='model-option',
                                              options=[{'label': 'Regresión Lineal', 'value': 'linear_regression'},
                                                       {'label': '240 Vecinos más cercanos', 'value': 240},
                                                       {'label': '120 Vecinos más cercanos', 'value': 120},
                                                       {'label': '60 Vecinos más cercanos', 'value': 60},
                                                       {'label': '10 Vecinos más cercanos', 'value': 10},
                                                       {'label': '2 Vecinos más cercanos', 'value': 2},
                                                       {'label': '1 Vecino más cercano', 'value': 1}],
                                              value='linear_regression')]),
                       html.Div([dcc.Graph(id='train',figure={}),
                                 dcc.Graph(id='test',figure={}),
                                 dcc.Graph(id='validation',figure={})],
                                style= {'width': '50%', 
                                        'display': 'flex'})])

@app.callback(
    [Output(component_id='train', component_property='figure'),
     Output(component_id='test', component_property='figure'),
     Output(component_id='validation', component_property='figure')],
    [Input(component_id='model-option', component_property='value')]
)
def regresion_graphs(model_option):
    
    if model_option =='linear_regression':
        x_modelo, y_modelo = crear_linea_regresion_lineal(datos_altura_peso['x_entrenamiento'], 
                                                          datos_altura_peso['y_entrenamiento'])
        
        nombre_modelo = 'Regresión Lineal'
    else:
        x_modelo, y_modelo = crear_linea_k_vecinos(datos_altura_peso['x_entrenamiento'], 
                                                   datos_altura_peso['y_entrenamiento'],
                                                   model_option)
        
        nombre_modelo = f'{str(model_option)}-Vecinos más cercanos'
        
    figuras = crear_figuras_entrenamiento_prueba_validacion(datos_altura_peso, 
                                                            x_modelo, 
                                                            y_modelo,
                                                            nombre_modelo)    

    return figuras['figura_entrenamiento'], figuras['figura_prueba'], figuras['figura_validacion']

In [4]:
app

#### Comparación de error entrenamiento, prueba, validación

In [5]:
app = JupyterDash(__name__)

df_errores = crear_df_error(datos_altura_peso)

# app
app.layout = html.Div([html.Div([html.H1("Demo Error de modelos de regresión"),
                                 dcc.Dropdown(id='model-options',
                                              options=[{'label': 'Entrenamiento', 'value': 'entrenamiento'},
                                                       {'label': 'Prueba', 'value': 'prueba'},
                                                       {'label': 'Validación', 'value': 'validacion'}],
                                              value=['entrenamiento'],
                                              multi=True)]),
                       html.Div([dcc.Graph(id='error',figure={})],
                                style= {'width': '50%', 
                                        'display': 'flex'})])

@app.callback(
    Output(component_id='error', component_property='figure'),
    [Input(component_id='model-options', component_property='value')]
)
def error_graphs(model_options):
    if not model_options:
        return {}
    figura_error = go.Figure()
    for tipo_de_error in model_options:
        figura_error = graficar_error(figura_error, tipo_de_error, df_errores) 

    return figura_error

In [6]:
app

## Ejemplo de Clasificación (Regresión Logística)

### Cargar Datos

In [7]:
data_titanic = pd.read_csv('./data/titanic_train.csv')

In [8]:
data_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Cargar aplicación

#### Visualizar conjunto de prueba

In [9]:
app = JupyterDash(__name__)

data_titanic['Sex'] = (data_titanic['Sex'] == 'male').astype(int)

# app
app.layout = html.Div([html.Div([html.H1("Demo Regresión Logística"),
                                 html.Div(children='Nota 1: Sólo se toman en cuenta las primeras dos variables seleccionadas'),
                                 html.Div(children='Nota 2: La primera variable tiene que ser continua (Edad o Tarífa)'),
                                 dcc.Dropdown(id='variable-options',
                                              options=[{'label': 'Variable 1: Tarifa', 'value': 'Fare'},
                                                       {'label': 'Variable 1: Edad', 'value': 'Age'},
                                                       {'label': 'Variable 1: Sexo', 'value': 'Sex'},
                                                       {'label': 'Variable 1: Clase del Pasajero', 'value': 'Pclass'}],
                                              value=['Fare', 'Age'],
                                              multi=True)]),
                       html.Div([dcc.Graph(id='test-one-variable', figure={}),
                                 dcc.Graph(id='test-two-variables', figure={})],
                                style={'width': '50%',
                                       'display': 'flex'})])


@app.callback(
    [Output(component_id='test-one-variable', component_property='figure'),
     Output(component_id='test-two-variables', component_property='figure')],
    [Input(component_id='variable-options', component_property='value')]
)
def logistic_regresion_graphs(variable_options):
    # crear input
    if not variable_options or len(variable_options) < 2:
        return [{}, {}]
    elif len(variable_options) > 2:
        variable_options = variable_options[0:2]
    if variable_options[0] not in ['Fare', 'Age']:
        return [{}, {}]


    data_titanic_na_filter = data_titanic[variable_options + ['Survived']]
    data_titanic_na_filter = data_titanic_na_filter.dropna()

    respuesta = data_titanic_na_filter.Survived
    variables = data_titanic_na_filter[variable_options]
    datos_titanic = dividir_datos(variables, respuesta)

    datos_logisticos = crear_linea_regresion_logistica(datos_titanic['x_entrenamiento'],
                                                       datos_titanic['y_entrenamiento'],
                                                       datos_titanic['x_prueba'],
                                                       variable_options[0])

    figura_una_variable, figura_dos_variables = crear_figuras_regresion_logistica(datos_logisticos, variable_options, .5)

    return figura_una_variable, figura_dos_variables

In [10]:
app